In [8]:
import torch; torch.__version__

'2.2.1+cu121'

In [1]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Loading the model

In [2]:
max_seq_length = 4096 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "allam_v0", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth: Will load allam_v0 as a legacy tokenizer.
allam_v0 does not have a padding token! Will use pad_token = <unk>.


Adding the LoRA adapter

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,   
    loftq_config = None, 
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


Data Prep

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset('json', data_files='sada/training_data/pretraining_data.jsonl')
dataset = dataset['train']

In [6]:
len(dataset)

158072

In [21]:
def count_tokens(examples):
    tokens = tokenizer(examples['text'], truncation=False, padding=False)  # Tokenize the text
    return {"num_tokens": [len(t) for t in tokens["input_ids"]]}  # Count tokens for each example

In [22]:
tokenized_dataset = dataset.map(count_tokens, batched=True)
total_tokens = sum(tokenized_dataset['num_tokens'])

Map:   0%|          | 0/158072 [00:00<?, ? examples/s]

In [23]:
print(f"Total number of tokens in the dataset: {total_tokens}")

Total number of tokens in the dataset: 3007497


In [7]:
EOS_TOKEN = tokenizer.eos_token

def combine_texts(example):

    combined_texts = []
    current_text = ""

    for i, text in enumerate(example['text']):
        current_text += text + "\n"

        if (i + 1) % 40 == 0:
            current_text = current_text.strip() + " " + EOS_TOKEN
            combined_texts.append(current_text)
            current_text = ""

    if current_text:
        current_text = current_text.strip() + " " + EOS_TOKEN
        combined_texts.append(current_text)

    return {"text": combined_texts}

In [8]:
dataset = dataset.map(combine_texts, batched=True)

In [9]:
len(dataset)

3952

In [10]:
print(dataset[1]['text'])
tokens = tokenizer(dataset[1]['text'], max_length= 4096,truncation=True, padding=False)
print(len(tokens['input_ids']))

Speaker2: <|Najdi|> زوجها لولدي هرشان </Najdi|>
Speaker1: <|Najdi|> ولدك هرسان </Najdi|>
Speaker1: <|Najdi|> ازوج حصة </Najdi|>
Speaker1: <|Najdi|> لولدك هرشان </Najdi|>
Speaker2: <|Najdi|> اذا تبي راحتك وخلاصك من المشاكل والمصايب </Najdi|>
Speaker1: <|Najdi|> بس </Najdi|>
Speaker1: <|Najdi|> اخاف ب اخاف ما تقبل بهرشان </Najdi|>
Speaker2: <|Najdi|> ومن قلك انها هي بتقبل بهرشان وغيره </Najdi|>
Speaker2: <|Najdi|> هي تقبل بميزر وبس </Najdi|>
Speaker1: <|Najdi|> اوه ميزر ميزر لا تجيبلي اسمه ولا اسم تاريه </Najdi|>
Speaker2: <|Najdi|> لازم يا ابو مسامح </Najdi|>
Speaker2: <|Najdi|> لازم اجيبلك تاريه علشان تفكر زين بمشكلة بنتك </Najdi|>
Speaker1: <|Najdi|> افكر </Najdi|>
Speaker1: <|Najdi|> وشلون افكر يا ابو هرشان </Najdi|>
Speaker1: <|Najdi|> شلون افكر وانا داري من ميزر طلع من الحبس والحين تلقاه جامعن زيد وقاعدين هم وام زيد و و شيخة وقاعدين يفكرون ويخططون شابوني زينو </Najdi|>
Speaker2: <|Najdi|> صحيح يخططون ويرسمون وشلون يقضون عليك الى الابد </Najdi|>
Speaker1: <|Najdi|> يقضوا علي للابد <

Continued Pretraining

In [10]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

In [11]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        
        max_steps = -1,
        warmup_steps = 10,
        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 2.5e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [12]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
6.451 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,952 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 247
 "-____-"     Number of trainable parameters = 844,169,216


Unsloth: Setting lr = 2.50e-05 instead of 5.00e-05 for embed_tokens.
Unsloth: Setting lr = 2.50e-05 instead of 5.00e-05 for lm_head.


Step,Training Loss
1,4.699000
2,4.705300
3,4.648300
4,4.364300
5,4.250200
6,3.741000
7,3.578800
8,3.444300
9,2.887500
10,3.082400


Saving the pretrained model

In [14]:
model.save_pretrained_merged("outputs/allam_v0.5_pretrained", tokenizer, save_method="merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 25.5 out of 60.46 RAM for saving.


100%|██████████| 32/32 [00:10<00:00,  2.99it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [3]:
max_seq_length = 4096 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "outputs/checkpoint-247", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth: Will load allam_v0 as a legacy tokenizer.
allam_v0 does not have a padding token! Will use pad_token = <unk>.
Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Inference

In [4]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(64008, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(64008, 4096)
          )
        )
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
        

In [5]:
inputs = tokenizer("Speaker1: <|Khaliji|> بدينا بالغيرة اللي ", return_tensors="pt").to("cuda")

In [6]:
outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
tokenizer.batch_decode(outputs)

['Speaker 1 :  <|Khaliji|> بدينا بالغيرة اللي  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|> ما هي طبيعية  </Khaliji|> \nSpeaker 1 :  <|Khaliji|>']